In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, balanced_accuracy_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_validate
from sklearn import preprocessing
from imblearn.under_sampling import RandomUnderSampler
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from sklearn.ensemble import VotingClassifier

!pip install rdkit
from rdkit import Chem
from rdkit.Chem import MACCSkeys


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 13.1 MB/s eta 0:00:00


In [3]:
def cf(cm):
    plt.rcParams["figure.figsize"] = (7, 5)          #Setting Plot Size
    cm_matrix = pd.DataFrame(data=cm,                #a pandas DataFrame created using the confusion matrix
                             columns=['0', '1'],     # The rows and columns are labeled '0' and '1',represent the two classes "biodegradable" ,"non-biodegradable")
                             index=['0', '1'])
    sns.heatmap(cm_matrix.astype('int'), annot=True, cmap='Greens', fmt="d",
               xticklabels=['NRB', 'RB'], yticklabels=['NRB', 'RB'], annot_kws={"size": 20})  # uses seaborn to generate a heatmap
    plt.tick_params(left=False, bottom=False)
    plt.xlabel('Predicted Class', fontsize=25)
    plt.xticks(fontsize=18)
    plt.ylabel('True Class', fontsize=25)
    plt.yticks(fontsize=18)
    plt.yticks(rotation=0)
    plt.show()

In [4]:
seed = 2021

In [5]:
def ensemb(vot='soft', pca=False, fp=True, scaling=True,
           undersampling=False, seed=123, test=0.2):     #This function builds ,evaluates an ensemble machine learning model to predict whether a molecule is biodegradable or not

    df = pd.read_csv('All-Public_dataset_Mordred.csv')  #df is DataFrame – it stores dataset as a table with rows and columns
    df = df.fillna(0)                                        #.fillna(0) replaces any missing values in the dataset with 0

    #defines 4 different ML models (KNN, SVM, Random Forest, Gradient Boosting),then combines them into one ensemble using a VotingClassifier.

    knn = KNeighborsClassifier(metric='manhattan', n_neighbors=5, weights='distance')
    svm = SVC(C=1, gamma=0.001, kernel='rbf', probability=True)
    rf = RandomForestClassifier(max_depth=8, min_samples_leaf=8, min_samples_split=16, n_estimators=100)
    gb = GradientBoostingClassifier(learning_rate=0.1, max_depth=4, n_estimators=75, subsample=0.7)

    votingC = VotingClassifier(estimators=[('knn', knn),
                                           ('svm', svm),
                                           ('rf',rf),
                                           ('gb', gb)], voting=vot, n_jobs=-1) #vot is a variable, which could either be 'hard' or 'soft'.

    if fp:              # checks whether fingerprint generation should be performed
        molfps = []     # molfps will store the molecular fingerprints for each molecule (SMILES string) in the dataset.
        for i in range(df.shape[0]):
            mol = (Chem.MolFromSmiles(df['SMILES'].iloc[i]))     # Convert SMILES to a molecule object by looping over all rows in dataset
            bitstring = list(MACCSkeys.GenMACCSKeys(mol).ToBitString())  #Generate MACCS keys for the molecule
            fp_vect = list(map(int, bitstring))   # Convert the bitstring to a list of integers
            molfps.append(fp_vect)                #Append the fingerprint vector to the list

        df['Fingerprint'] = molfps                #Add the fingerprint vectors as a new column in the dataframe

        x = df.iloc[:, 2:-1].to_numpy()           #Extract features (excluding the SMILES , class and Fingerprint columns)

        if pca:
            p = PCA(n_components=4)
            x = p.fit_transform(x)

        X = []
        for i in range(x.shape[0]):
            X.append(np.concatenate((x[i,:], np.array(df['Fingerprint'].iloc[i]))))
    if not fp:
        X = df.iloc[:, 2:].to_numpy()  #if fingerprint generation not requested then it takes all column starting from 3rd

    if scaling:
        X = preprocessing.StandardScaler().fit_transform(X)

    X = np.array(X)                   #converts list into a numpy array to be used easily by ML models
    y = df['Class'].to_numpy()        #Converts the target labels (the 'Class' column: biodegradable or non-biodegradable) into a NumPy array.

    if undersampling:
        print('Undersampling...')
        X, y = RandomUnderSampler(random_state=seed).fit_resample(X, y)

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test, shuffle=True, stratify=y, random_state=seed)

    votingC.fit(x_train, y_train)           #trains the model on training data
    y_pred = votingC.predict(x_test)        #Uses the trained model to predict class labels on the test data

    ba = balanced_accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test.tolist(), y_pred)
    sn = cm[1,1]/(cm[1,1]+cm[1,0])         #Measures how well the model detects positive cases
    sp = cm[0,0]/(cm[0,0]+cm[0,1])         #Measures how well the model detects negative cases
    er = (cm[0,1]+cm[1,0])/sum(sum(cm))

    return ba, sn, sp, er, cm

In [18]:
#selects the ML model, initializes the parameters and prepare the input data i.e the feature matrix X and the label class
def prepro(model, pca=False, scaling=True):         #model specifies which classifier to use

    df = pd.read_csv('All-Public_dataset_Mordred.csv')
    df = df.fillna(0)

    if model == 'KNN':
        df = pd.read_csv('All-Public_dataset_Mordred_GA_KNN.csv')
        df = df.fillna(0)
        clf = KNeighborsClassifier(metric='manhattan', n_neighbors=5, weights='distance') # initializes a KNN classifier with manhattan distance, 5 neighbours
    elif model == 'SVM':
        clf = SVC(C=1, gamma=0.001, kernel='rbf')
    elif model == 'RF':
        clf = RandomForestClassifier(max_depth=8, min_samples_leaf=8, min_samples_split=16, n_estimators=100)
    elif model == 'GB':
        clf = GradientBoostingClassifier(learning_rate=0.1, max_depth=4, n_estimators=75, subsample=0.7)

    molfps = []                          # Initialize an empty list to store molecular fingerprints
    for i in range(df.shape[0]):         # Loop through each row (molecule) in the dataset
        mol = (Chem.MolFromSmiles(df['SMILES'].iloc[i]))   # Convert the SMILES string into a molecular object
        bitstring = list(MACCSkeys.GenMACCSKeys(mol).ToBitString())    # Generate the MACCS fingerprint as a string of 0s and 1s
        fp_vect = list(map(int, bitstring))
        molfps.append(fp_vect)    # Add the fingerprint vector to the list

    df['Fingerprint'] = molfps    # Add the list of fingerprint vectors as a new column called 'Fingerprint' in the dataframe

    x = df.iloc[:, 2:-1].to_numpy()  # Extract all columns from the 3rd to the second last as a NumPy array (excluding SMILES and Class columns)

    X = []
    for i in range(x.shape[0]):
        X.append(np.concatenate((x[i,:], np.array(df['Fingerprint'].iloc[i]))))

    X = np.array(X)
    if scaling:
        X = preprocessing.StandardScaler().fit_transform(X)
    y = df['Class'].to_numpy()  #converts the "Class" column (which is a column in DataFrame) into a NumPy array (which is very similar to a list in Python)

    return clf, X, y   #clf: the selected machine learning model (KNN/SVM/RF/GB)

                       #X: the feature matrix (Mordred + Fingerprint, possibly scaled)

                      #y: the target labels

In [19]:
#Defines a function that train the model and evaluate it
def predict(clf, X, y, seed=123, test=0.2, undersampling=False):

    if undersampling:
        X, y = RandomUnderSampler(random_state=seed).fit_resample(X, y)

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test, shuffle=True, stratify=y, random_state=seed)  #Splits the dataset into training and testing sets

    clf.fit(x_train, y_train)          # trains the classifier using training data
    y_pred = clf.predict(x_test)       # predict the label of test set using trained model

    ba = balanced_accuracy_score(y_test, y_pred)     #out of all predictions(TP + TN + FP + FN), how many are actually correct(TP + TN)
    cm = confusion_matrix(y_test.tolist(), y_pred)   #creates a confusion matrix
    sn = cm[1,1]/(cm[1,1]+cm[1,0])                   #out of all actual positive cases (TP + FN), how many model predict as positive(TP) --> high
    sp = cm[0,0]/(cm[0,0]+cm[0,1])                   #out of all actual negatve cases (TN + FP), how many model predict as negative (TN)   --> high
    er = (cm[0,1]+cm[1,0])/sum(sum(cm))              #out of all predictions how many incorrect --> should be low

    return ba, sn, sp, er

In [20]:
import pandas as pd


In [21]:
from sklearn.svm import SVC


In [22]:
# Only using two test sizes: 0.2 and 0.8 to reduce computation
clf, X, y = prepro('SVM') #The SVM model and datset is first prepared

# Initialize empty lists to store the performance metrics for each test size and random seed
svm_ba_list = []
svm_sn_list = []
svm_sp_list = []
svm_er_list = []

# Reduced test size values: 0.2 and 0.8
#For each test size, it runs the evaluation 10 times with different random seeds (from 0 to 99 with step 10).
for t in tqdm([0.2, 0.8]):  # Only two test sizes
    ba_list = []
    sn_list = []
    sp_list = []
    er_list = []

    # Random seeds for reproducibility
    for r in range(0, 100, 10):  # Reduced the number of random seeds to 10
        ba, sn, sp, er = predict(clf, X, y, seed=r, test=t)   # Prediction and evaluation with the classifier
        ba_list.append(ba)                                    #Appends the results of each evaluation to their respective lists (ba_list, sn_list, sp_list, er_list)
        sn_list.append(sn)
        sp_list.append(sp)
        er_list.append(er)

    svm_ba_list.append(ba_list)
    svm_sn_list.append(sn_list)
    svm_sp_list.append(sp_list)
    svm_er_list.append(er_list)


  0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
clf, X, y = prepro('RF')

rf_ba_list = []
rf_sn_list = []
rf_sp_list = []
rf_er_list = []
for t in tqdm([0.2, 0.8]):
    ba_list = []
    sn_list = []
    sp_list = []
    er_list = []
    for r in range(0, 100, 10):
        ba, sn, sp, er = predict(clf, X, y, seed=r, test=t)
        ba_list.append(ba)
        sn_list.append(sn)
        sp_list.append(sp)
        er_list.append(er)
    rf_ba_list.append(ba_list)
    rf_sn_list.append(sn_list)
    rf_sp_list.append(sp_list)
    rf_er_list.append(er_list)

  0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
clf, X, y = prepro('GB')

gb_ba_list = []
gb_sn_list = []
gb_sp_list = []
gb_er_list = []
for t in tqdm([0.2, 0.8]):
    ba_list = []
    sn_list = []
    sp_list = []
    er_list = []
    for r in range(0, 100, 10):
        ba, sn, sp, er = predict(clf, X, y, seed=r, test=t)
        ba_list.append(ba)
        sn_list.append(sn)
        sp_list.append(sp)
        er_list.append(er)
    gb_ba_list.append(ba_list)
    gb_sn_list.append(sn_list)
    gb_sp_list.append(sp_list)
    gb_er_list.append(er_list)

  0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
enso_ba_list = []
enso_sn_list = []
enso_sp_list = []
enso_er_list = []
for t in tqdm([0.2, 0.8]):
    ba_list = []
    sn_list = []
    sp_list = []
    er_list = []
    for r in range(0, 100, 10):
        ba, sn, sp, er, cm = ensemb(vot='soft', seed=r, test=t)
        ba_list.append(ba)
        sn_list.append(sn)
        sp_list.append(sp)
        er_list.append(er)
    enso_ba_list.append(ba_list)
    enso_sn_list.append(sn_list)
    enso_sp_list.append(sp_list)
    enso_er_list.append(er_list)

  0%|          | 0/2 [00:00<?, ?it/s]

In [1]:
results_df = pd.DataFrame({'SVM_BA':svm_ba_list, 'SVM_Sn':svm_sn_list, 'SVM_Sp':svm_sp_list, 'SVM_ER':svm_er_list,
                           'RF_BA':rf_ba_list, 'RF_Sn':rf_sn_list, 'RF_Sp':rf_sp_list, 'RF_ER':rf_er_list,
                           'GB_BA':gb_ba_list, 'GB_Sn':gb_sn_list, 'GB_Sp':gb_sp_list, 'GB_ER':gb_er_list,
                           'ENSO_BA':enso_ba_list, 'ENSO_Sn':enso_sn_list, 'ENSO_Sp':enso_sp_list, 'ENSO_ER':enso_er_list})
results_df

NameError: name 'pd' is not defined

In [29]:
results_df.to_json('ML_random_test_Main.json', orient='table')
